In [1]:
import pandas as pd
from pyspark.mllib.recommendation import ALS, Rating

#Provide Path of ratings.csv dataset in "path" variable
path_rating="C:\\Program Files\\Git\\MyGitFilesDirectory\\data\\retail_db\\orders\\ml-latest-small\\ratings.csv"
path_movie="C:\\Program Files\\Git\\MyGitFilesDirectory\\data\\retail_db\\orders\\ml-latest-small\\movies.csv"

#Raw Data in RDD
rat=sc.textFile(path_rating)
mov=sc.textFile(path_movie)

#Transformation
movies=mov.map(lambda rec:rec.split(",")).map(lambda rec: (rec[0],rec[1]))
ratings=rat.map(lambda rec:rec.split(",")).map(lambda rec: (rec[0],rec[1],rec[2]))
header=ratings.first()
ratings=ratings.filter(lambda rec: rec!=header)

#DataFrame
ratingsDF = spark.createDataFrame(ratings)




In [2]:
#ALS in RDD
ratings = ratings.map(lambda l: Rating(int(l[0]), int(l[1]), float(l[2])))
rank = 50
numIterations = 10
userid,movieid=450,480
model = ALS.train(ratings, rank,numIterations,0.01 )
PredictFromDataFrame = model.predict(userid,movieid)

#ALS in DataFrame
model1 = ALS.train(ratingsDF, rank,numIterations,0.01 )
PredictFromRDD = model1.predict(userid,movieid)


In [3]:
moviename=movies.filter(lambda x: (x[0]!="movieId")).filter(lambda x:int(x[0])==movieid).first()[1]
print("Prediction :userid=",userid," movie=",moviename,"(using RDD)",PredictFromDataFrame ,"\n")
print("Prediction :userid=",userid," movie=",moviename,"(using DataFrame)", PredictFromRDD ,"\n")
print("Prediction :Difference(RDD-DataFrame)", PredictFromDataFrame-PredictFromRDD)

Prediction :userid= 450  movie= Jurassic Park (1993) (using RDD) 3.152263383520011 

Prediction :userid= 450  movie= Jurassic Park (1993) (using DataFrame) 2.862002668139766 

Prediction :Difference(RDD-DataFrame) 0.2902607153802448
